<a href="https://colab.research.google.com/github/pulkitsingh/Pokedex/blob/master/balanced_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mounting Google Drive

In [0]:
# mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


ValueError: ignored

In [0]:
cd gdrive/My\ Drive

/content/gdrive/My Drive


### Importing Libraries 

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, GlobalAveragePooling2D, BatchNormalization, Input, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam, RMSprop
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint
import os
from keras.models import load_model

Using TensorFlow backend.


### Loading Data

In [0]:
# directories for training and validation data
train_directory = 'balanced_data_clean/train'
validation_directory = 'balanced_data_clean/validation'

In [0]:
# making ImageDataGenerator instances for training and test data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                   zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
# loading the train and validation data
train_gen = train_datagen.flow_from_directory('balanced_data_clean/train',
                                              target_size=(224, 224),
                                              batch_size=32,
                                              color_mode="rgb",                                            
                                              class_mode='categorical', 
                                              shuffle=True, seed=333)

test_gen = test_datagen.flow_from_directory('balanced_data_clean/validation',
                                            target_size=(224, 224),
                                            color_mode = "rgb",
                                            batch_size=32,
                                            class_mode='categorical', 
                                            shuffle=True, seed=333)


Found 12903 images belonging to 149 classes.
Found 3317 images belonging to 149 classes.


### Specifying the model

In [0]:
# specifying parameters
batch_size = 10
input_shape = (10, 224, 224, 3)
num_training_samples = 12903
num_test_samples = 3317

In [0]:
# specifying the model

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(149))
model.add(Activation('softmax'))


In [0]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss=categorical_crossentropy, optimizer=adam, metrics=['accuracy'])
print(model.summary())



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 64)   

### Training the Model

In [0]:
# making a directory to save weights
import os
dir_name = 'pokemon_weights/train2'
#os.mkdir(dir_name)
path = os.path.join(os.getcwd(), dir_name)
print(path)

/content/gdrive/My Drive/pokemon_weights/train2


In [0]:
filepath = os.path.join(path, "model-{epoch:02d}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, 
                             save_best_only=False, save_weights_only=True, 
                             mode='auto', period=1)

In [0]:
history = model.fit_generator(train_gen, 
                    epochs=100,
                    workers=10,
                    use_multiprocessing=True,
                    shuffle=True,
                    callbacks=[checkpoint],
                    validation_data=test_gen,
                    validation_steps=1, 
                    verbose=1,
                    initial_epoch=86)

In [0]:
last_train = []
last_test = []
for i in range(86, 101):
  model.load_weights('/content/gdrive/My Drive/pokemon_weights/train2/model-{:02d}.hdf5'.format(i))
  train = model.evaluate_generator(train_gen, workers=10, 
                                   use_multiprocessing=True, verbose=1)
  test  = model.evaluate_generator(train_gen, workers=10, 
                                   use_multiprocessing=True, verbose=1)
  print("Train: {}, Test: {}")
  last_train.append(train)
  last_test.append(test)

  

In [0]:
np.argmax(history.history['val_acc'])
np.max(history.history['val_acc'])

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'], label="Training")
plt.plot(history.history['val_acc'], label="Validation", c='purple')
plt.ylim((0, 1))
plt.ylabel("Accuracy (percentage)")
plt.xlabel("Epoch")
plt.title("Classifier Accuracy for Original Dataset")
plt.legend()


In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label="Training")
plt.plot(history.history['val_loss'], label="Validation", c='purple')
plt.ylabel("Loss (Cross-Entropy)")
plt.xlabel("Epoch")
plt.title("Classifier Loss for Original Dataset")
plt.legend()

NameError: ignored

In [0]:
print(history.history)

### Visualizing Class-Wise Accuracy

In [0]:
import numpy as np

In [0]:
model.load_weights('/content/gdrive/My Drive/pokemon_weights/train2/model-90.hdf5')

In [0]:
# getting the class labels for the model
classes = train_gen.class_indices

In [0]:
# making wrapper folders in order to evaluate class-wise accuracy
# for each pokemon in the dataset
path = 'balanced_data_clean/validation'
for pokemon in os.listdir(path):
  if pokemon[0] != '.':

    # creating a wrapper folder in the directory
    wrapper_dir = os.path.join(path, "wrapper_{}".format(pokemon))
    os.mkdir(wrapper_dir)

    # moving the current folder to inside the wrapper folder
    cur_dir = os.path.join(path, pokemon)
    new_dir = os.path.join(wrapper_dir, pokemon)
    os.rename(cur_dir, new_dir)

In [0]:
# evaluating class-wise accuracy
accuracy = {}
counter = 0

# for each pokemon in the directory
for pokemon in os.listdir(path):
  if pokemon[0] != '.':

    # creating a data generator for just that class
    test_gen = test_datagen.flow_from_directory(path + '/{}'.format(pokemon),
                                            target_size=(224, 224),
                                            color_mode = "rgb",
                                            batch_size=32,
                                            class_mode='categorical', 
                                            shuffle=True, seed=333)
    
    # getting the output of the model on that class
    output = model.predict_generator(test_gen, workers=4, 
                                     use_multiprocessing=True, verbose=1)
    
    # getting prediction from class outputs
    preds = np.argmax(output, axis=1)

    # computing number of correct predictions
    pokemon_name = pokemon.replace("wrapper_", "")
    correct = np.count_nonzero(preds == classes[pokemon_name])

    # calculating accuracy as number of correct predictions/ total predictions
    acc = correct / np.size(preds)

    # saving accuracy in the dictionary
    accuracy[pokemon] = acc

    # printing results
    print(counter)
    counter += 1
    print("{}: {}. Correct = {}, Total = {}".format(pokemon, acc, correct, 
                                                    np.size(preds)))

    



In [0]:
print(accuracy.items())

dict_items([('wrapper_Rhydon', 0.42857142857142855), ('wrapper_Nidorina', 0.5652173913043478), ('wrapper_Lickitung', 0.68), ('wrapper_Farfetchd', 0.4), ('wrapper_Poliwhirl', 0.6363636363636364), ('wrapper_Charizard', 0.55), ('wrapper_Dragonite', 0.5714285714285714), ('wrapper_Nidoqueen', 0.3684210526315789), ('wrapper_Nidorino', 0.6363636363636364), ('wrapper_Cubone', 0.44), ('wrapper_MrMime', 0.5882352941176471), ('wrapper_Venomoth', 0.6086956521739131), ('wrapper_Exeggcute', 0.4090909090909091), ('wrapper_Paras', 0.3157894736842105), ('wrapper_Jynx', 0.8421052631578947), ('wrapper_Dratini', 0.6785714285714286), ('wrapper_Hitmonlee', 0.43333333333333335), ('wrapper_Mew', 0.43478260869565216), ('wrapper_Arbok', 0.7083333333333334), ('wrapper_Slowbro', 0.5238095238095238), ('wrapper_Grimer', 0.2857142857142857), ('wrapper_Caterpie', 0.85), ('wrapper_Kakuna', 0.6363636363636364), ('wrapper_Golem', 0.3888888888888889), ('wrapper_Nidoking', 0.45454545454545453), ('wrapper_Wartortle', 0.590

In [0]:
np.histogram(list(accuracy.values()))
import matplotlib.pyplot as plt

In [0]:
plt.hist(list(accuracy.values()), bins=10, histtype='bar')
plt.xlabel("Class-Wise Test Accuracy Ranges")
plt.ylabel("Number of Pokemon")
plt.title("Distribution of Class-Wise Test Accuracy")

### Exploring link Between Number of Training Image and Test Accuracy

In [0]:
# creating a dictionary with number of training examples per pokemon
counts = {}
for pokemon in os.listdir('balanced_data_clean/train'):
    if pokemon[0] != '.':
      name = pokemon.replace("wrapper_", "")
      counts[name] = len(os.listdir('balanced_data_clean/train/{}'.format(pokemon)))

In [0]:
print(counts.items())

In [0]:
# plotting number of training examples vs accuracy
acc, freq = [], []

for pokemon in os.listdir('balanced_data_clean/validation'):
  if pokemon[0] != '.':
    name = pokemon.replace("wrapper_", "")
    freq.append(counts[name])
    acc.append(accuracy[pokemon] * 100)

plt.scatter(freq, acc, c='purple')
plt.xlabel("Number of Training Examples")
plt.ylabel("Test Accuracy (percentage)")
plt.title("Number of Training Examples vs. Test Accuracy")